<a href="https://colab.research.google.com/github/josephaan/intellect-supreme/blob/master/WxGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install matplotlib
!pip install pandas
!pip install pathlib
!pip install torchinfo
!pip install tiktoken
import torch.nn.utils as utils
from torch.nn import functional as F
import torch.optim as optim
from torchinfo import summary
import tiktoken
import pandas as pd
from tqdm.auto import tqdm
from pathlib import Path
from typing import Optional, List
import torch
from torch import nn
import os
import math
torch.manual_seed(1337)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [ ]:
# dataset
files_no = 1000 # number of input text files to read
train_test_split = 0.85
block_size = 624 # number of tokens in one block. Maximum context sequence.
batch_size = 16 # size of the batch with blocks





In [ ]:
# Read and merge all data files
data_dir = Path("/content/drive/MyDrive/adam_s/data")
text_paths = list(data_dir.glob("*"))

text = ''

for file in tqdm(text_paths[:files_no]):
    with open(file, 'r', encoding='utf-8') as f:
        inp = f.read()
        text += inp

  0%|          | 0/19 [00:00<?, ?it/s]

In [ ]:
print("Dataset length: ", len(text))
print(text[30000:35000])

Dataset length:  440362672
Dirichlet Allocation (LDA)
	Hidden Markov models
	Deep Boltzmann machines
	VAEs
	GANs
	Naive Bayes classifiers, though named as a discriminative model, utilize Bayes' theorem to learn the joint distribution of X and Y under the assumption that the X variables are independent. Similarly, Gaussian mixture models describe the likelihood of a data point belonging to one of a group of normal distributions using the joint probability of the label and these distributions.
	LDA represents a document as the joint probability of a word and a set of underlying keyword lists (topics) that are used in a document. Hidden Markov models express the joint probability of a state and the next state of data, such as the weather on successive days of the week. As you will see in Chapter 4, Teaching Networks to Generate Digits, deep Boltzmann machines learn the joint probability of a label and the data vector it is associated with. The VAE and GAN models we will cover in Chapters 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(vocab_size)
print(chars)

to_int = { c:i for i,c in enumerate(chars) }
to_str = { i:c for i,c in enumerate(chars) }
encode = lambda s: [to_int[c] for c in s]
decode = lambda l: ''.join([to_str[i] for i in l])


data = torch.tensor(encode(text), dtype=torch.long)



954
['\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x93', '¡', '¢', '£', '¤', '¥', '¦', '§', '¨', '©', 'ª', '«', '¬', '\xad', '®', '¯', '°', '±', '²', '³', '´', 'µ', '¶', '·', '¸', '¹', 'º', '»', '¼', '½', '¾', '¿', 'À', 'Á', 'Â', 'Ã', 'Ä', 'Å', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ë', 'Ì', 'Í', 'Î', 'Ï', 'Ð', 'Ñ', 'Ò', 'Ó', 'Ô', 'Õ', 'Ö', '×', 'Ø', 'Ù', 'Ú', 'Û', 'Ü', 'Ý', 'Þ', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'õ', 'ö', '÷', 'ø', 'ù', 'ú', 'û', 'ü', 'ý', 'þ', 'ÿ', 'Ā', 'ā', 'ă', 'ą', '

In [ ]:
# Train/test data split
sep = int(train_test_split*len(data))
train_data = data[:sep]
test_data = data[sep:]

def random_batch(mode):
    data = train_data if mode == 'train' else test_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

X, Y = random_batch('train')


In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out


In [ ]:

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out


In [ ]:
'''
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)
'''
class FeedFoward(nn.Module):
    """ A complex feedforward neural network with multiple layers and non-linearities. """
    def __init__(self, n_embd, n_layers, dropout):
        super().__init__()
        self.net = nn.Sequential()
        for _ in range(n_layers):
            self.net.add_module(
                "linear", nn.Linear(n_embd, 8 * n_embd)
            )
            self.net.add_module("relu", nn.ReLU())
            self.net.add_module("dropout", nn.Dropout(dropout))
        self.net.add_module("output", nn.Linear(8 * n_embd, n_embd))

    def forward(self, x):
        return self.net(x)

#Block is the entire transformer except for the cross-attention/
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head, n_layer, dropout):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd, n_layer, dropout)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x



In [ ]:
class WxGPT(nn.Module):

    def __init__(self, vocab_size, n_embd, block_size, n_layer, n_head, dropout):
        super().__init__()
        self.vocab_size = vocab_size  # Store vocab_size as an instance variable
        self.block_size = block_size  # Store block_size as an instance variable

        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head, n_layer, dropout) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def top_k_sampling(self, logits, top_k):
        if top_k > 0:
            sorted_logits, sorted_indices = torch.sort(logits, descending=True)
            cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
            sorted_indices_to_remove = cumulative_probs > top_k
            sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
            sorted_indices_to_remove[..., 0] = 0
            indices_to_remove = sorted_indices[sorted_indices_to_remove]
            logits[indices_to_remove] = float('-inf')
        return logits

    def generate(self, text, max_new_tokens, temperature=1.0, top_k=None):
        # Encode the input text
        input_tokens = encode(text)  # You need to define the 'encode' function
        device = self.token_embedding_table.weight.device
        input_tokens_tensor = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(0)
        # Generate new tokens
        generated_tokens = self._generate_tokens(input_tokens_tensor, max_new_tokens, temperature, top_k)
        # Decode the generated tokens into text
        generated_text = decode(generated_tokens[0].tolist())  # You need to define the 'decode' function
        return generated_text

    def _generate_tokens(self, input_tokens, max_new_tokens, temperature, top_k):
        # Get the context tensor
        context = input_tokens.clone()
        # Generate new tokens
        for _ in range(max_new_tokens):
            # Crop context to the last block_size tokens
            context_cond = context[:, -self.block_size:]
            # Get the predictions
            logits, _ = self.forward(context_cond)
            # Focus only on the last time step
            logits = logits[:, -1, :].squeeze()
            # Apply temperature
            logits /= temperature
            # Apply top-k sampling if specified
            if top_k is not None:
                logits = self.top_k_sampling(logits, top_k)

            # Sample the next token
            next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
            # Append the next token to the context
            #context = torch.cat((context, next_token), dim=1)
            context = torch.cat((context, next_token.unsqueeze(1)), dim=1)

        return context


In [ ]:
#params
eval_interval = 2
eval_iters = 5
n_embd = 672
n_head = 16
n_layer = 12
dropout = 0.0

# training
epochs = 3000 # no. of epochs to train the model
eval_epochs = 100 # no. of epochs to calculate mean test loss
info_interval = 250 # test loss information frequency
learning_rate = 1e-4



# Check if the 'Block' class implementation is available and whether the tokenizer model file 'tokenizer.model' exists.

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = NgramLanguageModel(vocab_size, n_embd, block_size, n_layer, n_head, ngram_order, device)
model = WxGPT(vocab_size, n_embd, block_size, n_layer, n_head, dropout)
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')
summary(m)

110.196858 M parameters


Layer (type:depth-idx)                        Param #
WxGPT                                         --
├─Embedding: 1-1                              641,088
├─Embedding: 1-2                              419,328
├─Sequential: 1-3                             --
│    └─Block: 2-1                             --
│    │    └─MultiHeadAttention: 3-1           1,807,008
│    │    └─FeedFoward: 3-2                   7,231,392
│    │    └─LayerNorm: 3-3                    1,344
│    │    └─LayerNorm: 3-4                    1,344
│    └─Block: 2-2                             --
│    │    └─MultiHeadAttention: 3-5           1,807,008
│    │    └─FeedFoward: 3-6                   7,231,392
│    │    └─LayerNorm: 3-7                    1,344
│    │    └─LayerNorm: 3-8                    1,344
│    └─Block: 2-3                             --
│    │    └─MultiHeadAttention: 3-9           1,807,008
│    │    └─FeedFoward: 3-10                  7,231,392
│    │    └─LayerNorm: 3-11                   1,3

In [ ]:
@torch.no_grad()
def avg_loss():
    out = {}
    model.eval()
    for mode in ['train', 'test']:
        losses = torch.zeros(eval_epochs)
        for k in range(eval_epochs):
            X, Y = random_batch(mode)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[mode] = losses.mean()
    model.train()
    return out

In [ ]:
model_path = "/content/drive/MyDrive/adam_s/model_v0.01_mts_17.pth"

In [ ]:
def train(model, optimizer, epochs):

    results = {"train_loss": []}
    file_exists = os.path.exists(model_path)

    if file_exists == True:
        checkpoint = torch.load(model_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


    for epoch in tqdm(range(epochs)):

        if epoch % info_interval == 0 or epoch == epochs - 1:
            losses = avg_loss()
            print(f"Step {epoch}: train loss {losses['train']:.4f}, test loss {losses['test']:.4f}")
            torch.save({'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(),},'/content/drive/MyDrive/adam_s/model_v0.01_mts_17.pth')
            text_input = "I am"
            generated_text = model.generate(text_input, max_new_tokens=100, temperature=0.8, top_k=None)
            print(generated_text)

        X, Y = random_batch('train')

        logits, loss = model(X, Y)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Set an appropriate max_norm value
        optimizer.step()
        results["train_loss"].append(loss.item())

    print(f"Final train loss: {loss.item()}")
    torch.save({'model_state_dict': model.state_dict(),'optimizer_state_dict': optimizer.state_dict(),},'/content/drive/MyDrive/adam_s/model_v0.01_mts_17.pth')

    return results

In [ ]:
# Set the weight decay value
weight_decay = 1e-4  # You can adjust this value based on your specific needs

# Create the optimizer with weight decay
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)


In [ ]:
file_exists = os.path.exists(model_path)
print(file_exists)

True


In [ ]:
model_results = train(model=model,optimizer=optimizer, epochs=epochs)

  0%|          | 0/3000 [00:00<?, ?it/s]

Step 0: train loss 1.2120, test loss 1.3207
I am a great best to that altogether with a contusion. If you are mendid. But now, if you cannot be more
Step 250: train loss 1.2587, test loss 1.3972
I am at my existence of free Branden and the practical countrysiderations of the Africans;125 they raise


In [ ]:
ax = pd.DataFrame({'Train Loss': [loss for loss in model_results['train_loss']]}).plot(title='Train Loss Decrease', logy=True)

ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")

In [ ]:
text_input = "A man can"
generated_text = model.generate(text_input, max_new_tokens=4500, temperature=1, top_k=0.8)
print(generated_text)
